In [1]:
import numpy as np
import pandas as pd
import keras
import warnings
import import_ipynb
import Pipeline_Latest as pp2
import DeepBind_Latest as db22
import os
import sys
import matplotlib.pyplot as pl
import subprocess
warnings.filterwarnings("ignore")

dir(db22)

/home/abhinav/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


importing Jupyter notebook from Pipeline_Latest.ipynb
importing Jupyter notebook from DeepBind_Latest.ipynb


['__builtins__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 'deepbind1',
 'get_ipython',
 'import_ipynb',
 'json',
 'keras',
 'np',
 'one_hot_encode',
 'pd',
 'pp2',
 'warnings']

In [2]:
filenames1 = ['ENCFF127XXD_H3K4me3.bed','ENCFF099LMD_H3K4me2.bed', 'ENCFF139CKE_H4K20me1.bed', 'ENCFF183UQD_H3K4me1.bed','ENCFF322IFF_H3K27me3.bed', 'ENCFF379CMJ_EP300.bed', 'ENCFF498CMP_H3K36me3.bed', 'ENCFF578UBP_H3K27ac.bed', 'ENCFF624XRN_H2AFZ.bed', 'ENCFF737AMS_H3K4me3.bed', 'ENCFF894VEM_H3K9me3.bed']


In [3]:
filenames2 = ['ENCFF127XXD_H3K4me3.bed','ENCFF099LMD_H3K4me2.bed', 'ENCFF139CKE_H4K20me1.bed', 'ENCFF183UQD_H3K4me1.bed','ENCFF322IFF_H3K27me3.bed', 'ENCFF379CMJ_EP300.bed', 'ENCFF498CMP_H3K36me3.bed', 'ENCFF578UBP_H3K27ac.bed', 'ENCFF624XRN_H2AFZ.bed', 'ENCFF737AMS_H3K4me3.bed', 'ENCFF894VEM_H3K9me3.bed']


In [4]:
df_final = pd.DataFrame(columns=['File1','File2','Accuracy'])

print len(filenames1)
print len(filenames2)


11
11


In [6]:
from Bio import SeqIO
count=0
for i in range(len(filenames1)-1):
    for j in range(i+1,len(filenames2)):
        f1 = filenames1[i]
        f2 = filenames2[j]
        num_intersections_in30, num_in_class0, num_in_class1= pp2.pipe(f1,f2)
        
        file1, file_extension1 = os.path.splitext(f1)
        file2, file_extension2 = os.path.splitext(f2)
        
        class0_train= "inter_fa/"+file1+"_70"+".fa"
        class1_train= "inter_fa/"+file2+"_70"+".fa"
        class0_test= "inter_fa/"+file1+"_30"+".fa"
        class1_test= "inter_fa/"+file2+"_30"+".fa"
               

        list0=[]
        list1=[]

        #making dataframe for train data
        for record in SeqIO.parse(class0_train, "fasta"):
            list0.append(str(record.seq))
        for record in SeqIO.parse(class1_train, "fasta"):
            list1.append(str(record.seq))
        target=[]
    
    
        for t in range(len(list0)):
            target.append(0)
        
        for m in range(len(list1)):
            target.append(1)
        
        sequences=list0+list1

        dict = {'seq': sequences, 'target': target}
        df_train = pd.DataFrame(dict)
        
        
        #making dataframe for test data
        list0_test=[]
        list1_test=[]

        for record in SeqIO.parse(class0_test, "fasta"):
            list0_test.append(str(record.seq))
        for record in SeqIO.parse(class1_test, "fasta"):
            list1_test.append(str(record.seq))
        target=[]
    
    
        for t in range(len(list0_test)):
            target.append(0)
        
        for m in range(len(list1_test)):
            target.append(1)
        
        sequences1=list0_test+list1_test

        dict1 = {'seq': sequences1, 'target': target}
        df_test = pd.DataFrame(dict1)
        
        print(df_train.shape)
        print(df_test.shape)
        
        
        #calling deepbind
        accuracy,scores= db22.deepbind1(df_train,df_test,count)
        print("Num intersections in 30%: ",num_intersections_in30,"\n")
        print("Num in class0: ",num_in_class0,"\n")
        print("Num in class1: ",num_in_class1,"\n")
        print("File1: ",f1,"\n")
        print("File2: ",f2,"\n")
        print("Accuracy: ",accuracy,"\n")
        print("Prob: ",scores,"\n")
        #print("Probability: ",scores,"\n")
        df_final.loc[count,['File1']] = str(f1)
        df_final.loc[count, ['File2']] = str(f2)
        print accuracy
        df_final.loc[count, ['Accuracy']] = accuracy
        
        
        #histogram stuff
        title1="Class0 num: "+str(num_in_class0)+" Class1 num: "+str(num_in_class1)+" Num. of 30% intersections: "+str(num_intersections_in30)
        plot_title= "histograms/"+file1+"_"+file2+".png"
        pl.figure(figsize=(7,7))
#         n,bins,patches = pl.hist(scores,bins=10, density = False, edgecolor='black')
#         for p in patches:
#             pl.text(p.get_x() + p.get_width()/2., p.get_height(), '%d' % int(p.get_height()),fontsize=12, color='red', ha='center', va='bottom')
#         pl.title(title1)
#         pl.xlabel("Predicted probabilities")
#         pl.ylabel("Frequency")
        n1,bins1,patches1 = pl.hist(scores[0:num_in_class0],bins=10, density = False, edgecolor='black',alpha=0.5,label = "class0-red")
        n2,bins2,patches2 = pl.hist(scores[num_in_class0+1:],bins=10, density = False, edgecolor='black',alpha=0.5, label = "class1-black")
        for p in patches1:
            pl.text(p.get_x() + p.get_width()/2., p.get_height(), '%d' % int(p.get_height()),fontsize=12, color='red', ha='center', va='bottom')
        for p in patches2:
            pl.text(p.get_x() + p.get_width()/2., p.get_height(), '%d' % int(p.get_height()),fontsize=12, color='black', ha='center', va='bottom')
        pl.title(title1)
        pl.xlabel("Predicted probabilities")
        pl.ylabel("Frequency")
        pl.legend(loc='upper left')
        pl.savefig(plot_title)
        pl.clf()
        
        count=count+1

8333
(37158, 2)
(33033, 2)
('Shape is: ', (37158, 2))
[[[0. 0. 0. 1.]
  [0. 1. 0. 0.]
  [0. 1. 0. 0.]
  ...
  [0. 1. 0. 0.]
  [1. 0. 0. 0.]
  [1. 0. 0. 0.]]

 [[0. 0. 0. 1.]
  [0. 0. 0. 1.]
  [0. 1. 0. 0.]
  ...
  [1. 0. 0. 0.]
  [1. 0. 0. 0.]
  [1. 0. 0. 0.]]

 [[1. 0. 0. 0.]
  [0. 0. 0. 1.]
  [1. 0. 0. 0.]
  ...
  [0. 0. 0. 1.]
  [0. 0. 0. 1.]
  [0. 0. 0. 1.]]

 ...

 [[0. 0. 0. 1.]
  [0. 0. 0. 1.]
  [1. 0. 0. 0.]
  ...
  [0. 1. 0. 0.]
  [1. 0. 0. 0.]
  [1. 0. 0. 0.]]

 [[0. 1. 0. 0.]
  [0. 0. 1. 0.]
  [0. 0. 1. 0.]
  ...
  [1. 0. 0. 0.]
  [1. 0. 0. 0.]
  [1. 0. 0. 0.]]

 [[0. 0. 0. 1.]
  [0. 0. 1. 0.]
  [0. 0. 1. 0.]
  ...
  [0. 0. 0. 1.]
  [0. 1. 0. 0.]
  [1. 0. 0. 0.]]]
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_4 (Conv1D)            (None, 181, 10)           810       
Total params: 810
Trainable params: 810
Non-trainable params: 0
______________________________________________________

53348/53348 [==============================] - 9s 166us/step - loss: 0.5408 - acc: 0.7203
Epoch 4/5
53348/53348 [==============================] - 9s 165us/step - loss: 0.5319 - acc: 0.7276
Epoch 5/5
53348/53348 [==============================] - 9s 164us/step - loss: 0.5243 - acc: 0.7330
[[[0. 0. 0. 1.]
  [1. 0. 0. 0.]
  [0. 0. 1. 0.]
  ...
  [0. 0. 0. 1.]
  [0. 0. 0. 1.]
  [1. 0. 0. 0.]]

 [[0. 0. 1. 0.]
  [0. 0. 1. 0.]
  [1. 0. 0. 0.]
  ...
  [0. 1. 0. 0.]
  [1. 0. 0. 0.]
  [0. 1. 0. 0.]]

 [[1. 0. 0. 0.]
  [0. 0. 0. 1.]
  [0. 1. 0. 0.]
  ...
  [0. 1. 0. 0.]
  [1. 0. 0. 0.]
  [1. 0. 0. 0.]]

 ...

 [[1. 0. 0. 0.]
  [0. 0. 1. 0.]
  [0. 1. 0. 0.]
  ...
  [1. 0. 0. 0.]
  [1. 0. 0. 0.]
  [1. 0. 0. 0.]]

 [[0. 0. 1. 0.]
  [1. 0. 0. 0.]
  [1. 0. 0. 0.]
  ...
  [0. 0. 1. 0.]
  [0. 1. 0. 0.]
  [1. 0. 0. 0.]]

 [[0. 0. 1. 0.]
  [0. 1. 0. 0.]
  [0. 1. 0. 0.]
  ...
  [0. 1. 0. 0.]
  [0. 0. 1. 0.]
  [0. 1. 0. 0.]]]
23818/23818 [==============================] - 1s 52us/step
[[0.29764193]
 [0.28

('Num intersections in 30%: ', 3613, '\n')
('Num in class0: ', 13416, '\n')
('Num in class1: ', 34142, '\n')
('File1: ', 'ENCFF127XXD_H3K4me3.bed', '\n')
('File2: ', 'ENCFF183UQD_H3K4me1.bed', '\n')
('Accuracy: ', 0.7613970376082324, '\n')
('Prob: ', array([[0.37766   ],
       [0.3237612 ],
       [0.6025618 ],
       ...,
       [0.7886165 ],
       [0.87007564],
       [0.8476078 ]], dtype=float32), '\n')
0.7613970376082324
70
(82834, 2)
(35640, 2)
('Shape is: ', (82834, 2))
[[[0. 0. 0. 1.]
  [0. 0. 0. 1.]
  [1. 0. 0. 0.]
  ...
  [0. 0. 1. 0.]
  [1. 0. 0. 0.]
  [0. 0. 1. 0.]]

 [[0. 0. 0. 1.]
  [0. 0. 1. 0.]
  [0. 0. 1. 0.]
  ...
  [1. 0. 0. 0.]
  [0. 1. 0. 0.]
  [0. 0. 0. 1.]]

 [[0. 0. 1. 0.]
  [1. 0. 0. 0.]
  [0. 0. 1. 0.]
  ...
  [0. 1. 0. 0.]
  [0. 0. 0. 1.]
  [0. 1. 0. 0.]]

 ...

 [[0. 1. 0. 0.]
  [0. 0. 0. 1.]
  [0. 0. 1. 0.]
  ...
  [0. 0. 0. 1.]
  [0. 0. 0. 1.]
  [0. 1. 0. 0.]]

 [[0. 1. 0. 0.]
  [0. 0. 1. 0.]
  [1. 0. 0. 0.]
  ...
  [1. 0. 0. 0.]
  [1. 0. 0. 0.]
  [0. 0. 

Epoch 1/5
149228/149228 [==============================] - 25s 169us/step - loss: 0.3789 - acc: 0.8573
Epoch 2/5
149228/149228 [==============================] - 24s 164us/step - loss: 0.3636 - acc: 0.8581
Epoch 3/5
149228/149228 [==============================] - 25s 167us/step - loss: 0.3560 - acc: 0.8598
Epoch 4/5
149228/149228 [==============================] - 25s 168us/step - loss: 0.3503 - acc: 0.8619
Epoch 5/5
149228/149228 [==============================] - 25s 167us/step - loss: 0.3448 - acc: 0.8638
[[[0. 0. 0. 1.]
  [1. 0. 0. 0.]
  [0. 0. 1. 0.]
  ...
  [0. 0. 0. 1.]
  [0. 0. 0. 1.]
  [1. 0. 0. 0.]]

 [[0. 0. 1. 0.]
  [0. 0. 1. 0.]
  [1. 0. 0. 0.]
  ...
  [0. 1. 0. 0.]
  [1. 0. 0. 0.]
  [0. 1. 0. 0.]]

 [[1. 0. 0. 0.]
  [0. 0. 0. 1.]
  [0. 1. 0. 0.]
  ...
  [0. 1. 0. 0.]
  [1. 0. 0. 0.]
  [1. 0. 0. 0.]]

 ...

 [[0. 1. 0. 0.]
  [0. 0. 0. 1.]
  [1. 0. 0. 0.]
  ...
  [0. 0. 1. 0.]
  [0. 0. 1. 0.]
  [0. 1. 0. 0.]]

 [[0. 0. 0. 1.]
  [1. 0. 0. 0.]
  [0. 1. 0. 0.]
  ...
  [0. 0. 

('Num intersections in 30%: ', 446, '\n')
('Num in class0: ', 13416, '\n')
('Num in class1: ', 27926, '\n')
('File1: ', 'ENCFF127XXD_H3K4me3.bed', '\n')
('File2: ', 'ENCFF498CMP_H3K36me3.bed', '\n')
('Accuracy: ', 0.7801324498308866, '\n')
('Prob: ', array([[0.7543704 ],
       [0.3741249 ],
       [0.2579821 ],
       ...,
       [0.79932106],
       [0.48349437],
       [0.65974295]], dtype=float32), '\n')
0.7801324498308866
630
(72211, 2)
(43871, 2)
('Shape is: ', (72211, 2))
[[[0. 0. 0. 1.]
  [0. 0. 0. 1.]
  [1. 0. 0. 0.]
  ...
  [0. 0. 1. 0.]
  [1. 0. 0. 0.]
  [0. 0. 1. 0.]]

 [[0. 0. 0. 1.]
  [0. 0. 1. 0.]
  [0. 0. 1. 0.]
  ...
  [1. 0. 0. 0.]
  [0. 1. 0. 0.]
  [0. 0. 0. 1.]]

 [[0. 1. 0. 0.]
  [1. 0. 0. 0.]
  [0. 0. 1. 0.]
  ...
  [1. 0. 0. 0.]
  [0. 0. 1. 0.]
  [0. 0. 1. 0.]]

 ...

 [[0. 0. 1. 0.]
  [0. 0. 1. 0.]
  [0. 1. 0. 0.]
  ...
  [0. 0. 0. 1.]
  [0. 1. 0. 0.]
  [1. 0. 0. 0.]]

 [[0. 0. 0. 1.]
  [1. 0. 0. 0.]
  [0. 1. 0. 0.]
  ...
  [0. 1. 0. 0.]
  [0. 1. 0. 0.]
  [0. 1.

...

<Figure size 504x504 with 0 Axes>

<Figure size 504x504 with 0 Axes>

<Figure size 504x504 with 0 Axes>

<Figure size 504x504 with 0 Axes>

<Figure size 504x504 with 0 Axes>

In [7]:
print df_final

                       File1                     File2  Accuracy
0    ENCFF127XXD_H3K4me3.bed   ENCFF099LMD_H3K4me2.bed   0.64166
1    ENCFF127XXD_H3K4me3.bed  ENCFF139CKE_H4K20me1.bed  0.767099
2    ENCFF127XXD_H3K4me3.bed   ENCFF183UQD_H3K4me1.bed  0.761397
3    ENCFF127XXD_H3K4me3.bed  ENCFF322IFF_H3K27me3.bed  0.778297
4    ENCFF127XXD_H3K4me3.bed     ENCFF379CMJ_EP300.bed   0.74504
5    ENCFF127XXD_H3K4me3.bed  ENCFF498CMP_H3K36me3.bed  0.780132
6    ENCFF127XXD_H3K4me3.bed   ENCFF578UBP_H3K27ac.bed  0.755473
7    ENCFF127XXD_H3K4me3.bed     ENCFF624XRN_H2AFZ.bed  0.605159
8    ENCFF127XXD_H3K4me3.bed   ENCFF737AMS_H3K4me3.bed  0.751251
9    ENCFF127XXD_H3K4me3.bed   ENCFF894VEM_H3K9me3.bed  0.808993
10   ENCFF099LMD_H3K4me2.bed  ENCFF139CKE_H4K20me1.bed  0.687803
11   ENCFF099LMD_H3K4me2.bed   ENCFF183UQD_H3K4me1.bed  0.689985
12   ENCFF099LMD_H3K4me2.bed  ENCFF322IFF_H3K27me3.bed   0.65814
13   ENCFF099LMD_H3K4me2.bed     ENCFF379CMJ_EP300.bed  0.768581
14   ENCFF099LMD_H3K4me2.

In [8]:
df_final.to_csv("FinalAccuracies_withmodel.csv")


In [9]:
print df_final


                       File1                     File2  Accuracy
0    ENCFF127XXD_H3K4me3.bed   ENCFF099LMD_H3K4me2.bed   0.64166
1    ENCFF127XXD_H3K4me3.bed  ENCFF139CKE_H4K20me1.bed  0.767099
2    ENCFF127XXD_H3K4me3.bed   ENCFF183UQD_H3K4me1.bed  0.761397
3    ENCFF127XXD_H3K4me3.bed  ENCFF322IFF_H3K27me3.bed  0.778297
4    ENCFF127XXD_H3K4me3.bed     ENCFF379CMJ_EP300.bed   0.74504
5    ENCFF127XXD_H3K4me3.bed  ENCFF498CMP_H3K36me3.bed  0.780132
6    ENCFF127XXD_H3K4me3.bed   ENCFF578UBP_H3K27ac.bed  0.755473
7    ENCFF127XXD_H3K4me3.bed     ENCFF624XRN_H2AFZ.bed  0.605159
8    ENCFF127XXD_H3K4me3.bed   ENCFF737AMS_H3K4me3.bed  0.751251
9    ENCFF127XXD_H3K4me3.bed   ENCFF894VEM_H3K9me3.bed  0.808993
10   ENCFF099LMD_H3K4me2.bed  ENCFF139CKE_H4K20me1.bed  0.687803
11   ENCFF099LMD_H3K4me2.bed   ENCFF183UQD_H3K4me1.bed  0.689985
12   ENCFF099LMD_H3K4me2.bed  ENCFF322IFF_H3K27me3.bed   0.65814
13   ENCFF099LMD_H3K4me2.bed     ENCFF379CMJ_EP300.bed  0.768581
14   ENCFF099LMD_H3K4me2.